In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import models

In [2]:
model = models.load_model("./model")

2024-04-09 20:07:35.982667: I tensorflow/core/platform/profile_utils/cpu_utils.cc:101] CPU Frequency: 2983000000 Hz
2024-04-09 20:07:35.982998: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1725fb450 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-04-09 20:07:35.983024: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [3]:
'''
Takes two (20, 20) numpy arrays (meanOutTransit representing mean out of transit image, diffImage representing difference image).

Returns input values for model.
'''
def get_inputs(meanOutTransit, diffImage):
    stacked = np.array([diffImage, meanOutTransit])
    stacked_mins = np.array([np.amin(image) for image in stacked])
    stacked_maxs = np.array([np.amax(image) for image in stacked])

    stacked_norm = np.array([(image - im_min) / (im_max - im_min) 
                                   for image, im_min, im_max in zip(stacked, stacked_mins, stacked_maxs)])
    diff_btwn_norm_images = stacked_norm[0] - stacked_norm[1]
    min_diff_btwn_images = np.amin(diff_btwn_norm_images)
    max_diff_btwn_images = np.amax(diff_btwn_norm_images)

    final_stacked_mins = np.append(stacked_mins, min_diff_btwn_images)
    final_stacked_maxs = np.append(stacked_maxs, max_diff_btwn_images)
    final_stacked_norm = np.append(stacked_norm,
                                   [(diff_btwn_norm_images - min_diff_btwn_images) / (max_diff_btwn_images - min_diff_btwn_images)],
                                   axis=0)
    mins_maxs = np.concatenate((final_stacked_mins, final_stacked_maxs))
    
    train_maxs = np.array([ 6.94631623e+01,  3.47234857e+03, -1.65238096e-02,  4.24696856e+03, 1.25791510e+05,  9.98975425e-01])
    train_mins = np.array([-4.87401453e+03, -7.80439165e+04, -1.00000000e+00, -4.82699944e+03, 9.95267428e+00,  2.60097914e-03])
    
    mins_maxs_norm = (mins_maxs - train_mins) / (train_maxs - train_mins)
    return [image.reshape(1, 20, 20, 1) for image in final_stacked_norm] + [mins_maxs_norm.reshape(1, 6)]

In [4]:
np.random.seed(0)
meanOutTransit = np.random.rand(20, 20)
diffImage = np.random.rand(20, 20)

In [5]:
inputs = get_inputs(meanOutTransit, diffImage)
inputs

[array([[[[0.40071981],
          [0.92986577],
          [0.09843883],
          [0.94590966],
          [0.86993671],
          [0.45373432],
          [0.32600389],
          [0.2318489 ],
          [0.61437483],
          [0.0317581 ],
          [0.01425272],
          [0.42831413],
          [0.06683143],
          [0.25108626],
          [0.22024125],
          [0.25233911],
          [0.12994546],
          [0.01067535],
          [0.11434168],
          [0.61839886]],
 
         [[0.97492543],
          [0.99104816],
          [0.40853085],
          [0.16191196],
          [0.63872315],
          [0.48995352],
          [0.99011096],
          [0.06405572],
          [0.78350064],
          [0.28762069],
          [0.2405417 ],
          [0.66251605],
          [0.24519606],
          [0.66587768],
          [0.51701367],
          [0.42359746],
          [0.55447182],
          [0.28627087],
          [0.70667917],
          [0.41434586]],
 
         [[0.35991997],
          

In [6]:
model.predict(inputs)

array([[0.05503707]], dtype=float32)